# Analyze and Transform

## Import

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import numpy as np
import os

In [2]:
SQL_FILE = "../temp/meine_datenbank.db"
TEMP_PATH = os.path.join("..", "temp")

In [3]:
absolute_path = os.path.abspath(SQL_FILE)
print("Absoluter Pfad zur Datenbank:", absolute_path)

# Überprüfe ob die Datei am angegebenen Pfad existiert
if os.path.exists(absolute_path):
    print("Die Datei existiert. Verbindung wird hergestellt.")
    con = sqlite3.connect(absolute_path)
else:
    print("Die Datei existiert nicht am angegebenen Pfad. Bitte überprüfen Sie den Pfad.")

Absoluter Pfad zur Datenbank: /Users/dennis/Desktop/DBU/ads-03/ads-03-Studienarbeit/temp/meine_datenbank.db
Die Datei existiert. Verbindung wird hergestellt.


In [4]:
con = sqlite3.connect(SQL_FILE)

In [5]:
partei_df=pd.read_sql_query("select * from party", con=con)
wahlen_df=pd.read_sql_query("select * from wahlen", con=con)

# Analyse

In [6]:
print(partei_df.shape, wahlen_df.shape)

(74966, 4) (64, 7)


In [7]:
partei_df.head()

,date,count,word,paper
0,2022-04-18,1,linke,zeit
1,2022-04-18,2,spd,faz
2,2022-04-18,2,cdu,faz
3,2022-04-18,1,afd,faz
4,2022-04-18,1,cdu,faz


In [8]:
wahlen_df.head()

,date,CDU,SPD,Grüne,FDP,Linke,AfD
0,2021-01-15 00:00:00.000000,37,15,20,5,8,10
1,2021-01-28 00:00:00.000000,37,15,20,6,7,9
2,2021-02-26 00:00:00.000000,35,16,19,7,7,10
3,2021-03-26 00:00:00.000000,28,15,23,9,7,12
4,2021-04-16 00:00:00.000000,31,14,21,9,7,11


In [9]:
# Eine Kopie der Daten erstellen
wahlen = wahlen_df.copy()
wordcount = partei_df.copy()

## Wordcount df erstellen

In diesem Code werden die gesuchten Parteinamen ab einem bestimmten Datum gefiltert, aggregiert und in verschiedene Zeitformate umgewandelt, um eine übersichtliche Auswertung zu ermöglichen.

In [10]:
# Einschränken auf Daten ab 2021-04-01 und Index zurücksetzen
wordcount = wordcount[wordcount['date'] >= '2021-04-01'].reset_index(drop=True)
# Gruppieren nach Datum und Wort sowie Aufsummieren der 'count'-Werte
wordcount = wordcount.groupby(['date', 'word']).agg({'count': 'sum'}).reset_index()

# Konvertieren der 'date'-Spalte in ein Datumsformat
wordcount["date"] = pd.to_datetime(wordcount["date"])

# Hinzufügen einer Spalte für das Jahr und den Monat (Format 'YYYY-MM')
wordcount["month"] = wordcount["date"].dt.strftime('%Y-%m')
# Hinzufügen einer Spalte für das Quartal
wordcount["quarter"] = wordcount["date"].dt.to_period('Q')

# Ermitteln der Gesamtanzahl pro Wort
word_counts = wordcount.groupby("word")["count"].sum()

In [13]:
word_counts.sort_values()

word
afd       14775
linke     15054
fdp       15077
grünen    18473
spd       21914
cdu       22224
Name: count, dtype: int64

In [14]:
wordcount.head()

,date,word,count,month,quarter
0,2021-04-01,afd,8,2021-04,2021Q2
1,2021-04-01,cdu,43,2021-04,2021Q2
2,2021-04-01,fdp,2,2021-04,2021Q2
3,2021-04-01,grünen,20,2021-04,2021Q2
4,2021-04-01,linke,9,2021-04,2021Q2


In [15]:
# Liste der Wörter, die behalten werden sollen
words_to_keep = ["grünen", "afd"]
wordkeep = ["spd", "cdu"]
# Filtern des DataFrames, um nur die gewünschten Wörter zu behalten
kleine_parties = wordcount[wordcount["word"].isin(words_to_keep)]
große_parties = wordcount[wordcount["word"].isin(wordkeep)]

## Wahlen (Barometer) df erstellen

In diesem Code werden die Barometerdaten auf einen bestimmten Zeitraum eingeschränkt, ihre Spalten vereinheitlicht und zusätzliche Zeitintervalle für die Analyse hinzugefügt.

In [16]:
# Konvertieren der 'date'-Spalte in ein Datumsformat
wahlen["date"] = pd.to_datetime(wahlen["date"])

# Daten auf Zeitraum vom 01.04.2021 bis 01.02.2024 beschränken
wahlen = wahlen[wahlen["date"] >= "2021-04-01"].reset_index(drop=True)
wahlen = wahlen[wahlen["date"] <= "2024-02-01"].reset_index(drop=True)

# Spaltennamen vereinheitlichen und neue Zeitspalten hinzufügen
wahlen.columns = wahlen.columns.str.lower()
wahlen["quarter"] = wahlen["date"].dt.to_period("Q").astype(str)
wahlen["month"] = wahlen["date"].dt.to_period("M").astype(str)

# Konvertieren der CDU-Spalte in Integer
wahlen["cdu"] = wahlen["cdu"].astype(int)

# Länge jeder Spalte ausgeben
for key, value in wahlen.items():
    print(f'Länge von {key}:', len(value))


In [21]:
wahlen.head()

,date,cdu,spd,grüne,fdp,linke,afd,quarter,month
0,2021-04-16,31,14,21,9,7,11,2021Q2,2021-04
1,2021-05-07,25,14,26,10,7,11,2021Q2,2021-05
2,2021-05-21,24,14,25,11,7,11,2021Q2,2021-05
3,2021-06-10,28,15,22,10,7,11,2021Q2,2021-06
4,2021-06-25,29,14,22,10,7,10,2021Q2,2021-06


## Export

In [22]:
kleine_parties.to_pickle(os.path.join(TEMP_PATH, "kleine.pickle"))

In [23]:
große_parties.to_pickle(os.path.join(TEMP_PATH, "große.pickle"))

In [24]:
wahlen.to_pickle(os.path.join(TEMP_PATH, "wahlen.pickle"))